# Panopto Data Analysis

Panopto gives us "Stream Source" in their "Sessions created or edited" report, but usage data (including date last viewed) are in the Session usage report. Below, I use the pandas data manipulation library to stitch together these two data sources and compile some statistics.

In [183]:
import pandas as pd

sessions_file = 'sessionscreatedoredited.csv'
usage_file = 'sessionusage.csv'
sessions = pd.read_csv(sessions_file, low_memory=False).set_index('Session ID')
usage = pd.read_csv(usage_file, low_memory=False).set_index('Session ID')
# trim redundant columns from usage so we can cleanly join them
usage = usage.drop([c for c in sessions.columns if c in usage.columns], axis='columns')
# convert fields to datetimes
usage['Most Recent View Date'] = pd.to_datetime(usage['Most Recent View Date'])
sessions['Start Time'] = pd.to_datetime(sessions['Start Time'])
data = sessions.join(usage)

## Total Hours

In [184]:
# derivative columns
data['hours'] = data['Session Length'] / 60
data['Root Folder'] = data['Root Folder (Level 0)']
# @TODO a "lowest-level folder" of sorts might be useful but complicated to code
data['folder path'] = '/' + data['Root Folder (Level 0)'] + '/' + data['Subfolder (Level 1)'].astype(str) + '/' + data['Subfolder (Level 2)'].astype(str) + '/' + data['Subfolder (Level 3)'].astype(str) + '/' + data['Subfolder (Level 4)'].astype(str)

In [185]:
table = data[['Stream Source', 'hours']].groupby(['Stream Source']).sum().sort_values('hours', ascending=False)
table.loc['Total'] = table.sum()
table.reset_index()

,Stream Source,hours
0,Zoom,16169.953430
1,Web API,3188.285008
2,Panopto for Mac,134.287582
3,Panopto for Windows,71.140858
4,Panopto Capture,48.810205
5,Mixed,5.244231
6,RTMP,4.475007
7,iOS,3.401765
8,Unspecified,1.119587
9,Android,0.414145


In [186]:
table = data[['Root Folder', 'hours']].groupby(['Root Folder']).sum().sort_values('hours', ascending=False)
table.reset_index()

,Root Folder,hours
0,Users,15941.912690
1,Moodle,3454.757359
2,Libraries,154.739848
3,CCA Departments,65.413153
4,Tutorials,6.732931
5,First Year Program,3.375854


### Unwatched Hours by Source

In [187]:

unwatched = data[data['Views and Downloads'].isnull()]
table = unwatched[['Stream Source', 'hours']].groupby(['Stream Source']).sum().sort_values('hours', ascending=False)
table.loc['Total'] = table.sum()
table.reset_index()

,Stream Source,hours
0,Zoom,13313.311031
1,Web API,513.479401
2,Panopto for Mac,15.085160
3,Panopto Capture,5.897811
4,Panopto for Windows,5.234186
5,iOS,1.794449
6,Mixed,1.686691
7,Android,0.252345
8,RTMP,0.000000
9,Unspecified,0.000000


"Web API" refers to uploads via the website. See the list of "Stream Source" values on their reports documentation page: https://support.panopto.com/s/article/System-Usage-Report-Fields

### Unwatched Hours By Root Folder

In [188]:
unwatched[['Root Folder', 'hours']].groupby(['Root Folder']).sum().sort_values('hours', ascending=False).reset_index()

,Root Folder,hours
0,Users,13155.459902
1,Moodle,682.489447
2,Libraries,16.602274
3,CCA Departments,1.927585
4,Tutorials,0.261867


Most unwatched videos come from Zoom and are in User folders. There's still a significant amount of unwatched videos in the Moodle folder hierarchy, though.

### Watched Hours by Source

In [189]:

watched = data[data['Views and Downloads'] >= 1]
table = watched[['Stream Source', 'hours']].groupby(['Stream Source']).sum().sort_values('hours', ascending=False)
table.loc['Total'] = table.sum()
table.reset_index()

,Stream Source,hours
0,Zoom,2856.642399
1,Web API,2674.805607
2,Panopto for Mac,119.202422
3,Panopto for Windows,65.906672
4,Panopto Capture,42.912394
5,RTMP,4.475007
6,Mixed,3.557541
7,iOS,1.607316
8,Unspecified,1.119587
9,Android,0.161800


### Watched Hours by Root Folder

In [190]:
watched[['Root Folder', 'hours']].groupby(['Root Folder']).sum().sort_values('hours', ascending=False).reset_index()

,Root Folder,hours
0,Users,2786.452788
1,Moodle,2772.267913
2,Libraries,138.137575
3,CCA Departments,63.485569
4,Tutorials,6.471065
5,First Year Program,3.375854


A few conclusions:
  - Zoom is the source of the majority of hours (≈80%) but much Zoom content (79%) goes unwatched
  - Zoom recordings tend to go into User folders and thus User folders hold most of the unwatched content
  - Content uploaded by other sources is almost always watched at least once. Despite Zoom making up far more hours, uploaded ("Web API") sessions were watched almost as much. There's only a trace amount of content from other sources (like the Panopto desktop and web apps) and this content is almost always watched.
  - Similar to how non-Zoom content makes up almost as much watched content despite being a small share of our overall storage, sessions in Moodle folders were watched almost as much as the far-more-numerous sessions in Users folders

----

Now we want to look at videos that were watched at least once, but not recently.

In [191]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

# @TODO update this date if new reports are downloaded
report_date = datetime(year=2022, month=6, day=1)
six_months_ago = report_date - relativedelta(months=6)
one_year_ago = report_date - relativedelta(years=1)
eighteen_mo_ago = report_date - relativedelta(months=18)

watched_6_months = data[data['Most Recent View Date'] > six_months_ago]
watched_12_months = data[data['Most Recent View Date'] > one_year_ago]
watched_18_months = data[data['Most Recent View Date'] > eighteen_mo_ago]

print("Hours watched in the last...")
print("6 months:", round(watched_6_months['hours'].sum(), 2))
print("12 months:", round(watched_12_months['hours'].sum(), 2))
print("18 months:", round(watched_18_months['hours'].sum(), 2))

Hours watched in the last...
6 months: 1761.83
12 months: 3104.84
18 months: 4835.15


## Hours from last Academic Year

One last question: how many hours of various sorts were added in the 2021-22 academic year? This helps us anticipate how much content will be added in the upcoming year.

In [192]:
beginning_ay = datetime(year=2021, month=6, day=1)
ending_ay = datetime(year=2022, month=6, day=1)
ly_data = data[(data['Start Time'] >= beginning_ay) & (data['Start Time'] < ending_ay)]
table = ly_data[['Stream Source', 'hours']].groupby(['Stream Source']).sum().sort_values('hours', ascending=False)
table.loc['Total'] = table.sum()
table.reset_index()

,Stream Source,hours
0,Zoom,5665.411574
1,Web API,1005.946075
2,Panopto for Mac,30.158516
3,Panopto for Windows,13.988797
4,Panopto Capture,13.542681
5,iOS,3.401765
6,Android,0.414145
7,Mixed,0.047321
8,RTMP,0.000000
9,Unspecified,0.000000


In [193]:
table = ly_data[ly_data['Most Recent View Date'] >= beginning_ay]
table = table[['Stream Source', 'hours']].groupby(['Stream Source']).sum().sort_values('hours', ascending=False)
table.loc['Total'] = table.sum()
print('Watched Hours from last year')
table.reset_index()

Watched Hours from last year


,Stream Source,hours
0,Zoom,1149.595661
1,Web API,821.595582
2,Panopto for Mac,22.547846
3,Panopto for Windows,13.356247
4,Panopto Capture,10.529409
5,iOS,1.607316
6,Android,0.161800
7,Mixed,0.047321
8,Total,2019.441183


## Potential Plans

**The Good News**: since only 5770 hours have been watched ever, we can focus on plans that do not need a storage hours bundle since the default (8800) should give us enough room.

**The Bad News**: the 4,400 created hours bucket is lower than the amount of content we've created every year so far, so we may be forced to reduce our incoming hours which basically necessitates turning off the Zoom integration. We actually add _more unwatched Zoom hours_ each year than the created hours bucket.

Below, four scenarios are presented.

1. Saving money is important - no added hours
2. Saving existing content is important - only add a bundle of archived hours
3. Retaining Zoom recordings is important - only add a bundle of created hours
4. We can afford to import Zooms _and_ pay for an increased archive

These plans do not look to spend money on increasing stored hours. We have much unwatched content; we are incentivized to create a retention plan that minimizes paying for that. Therefore the plans above investigate adding an hours bundle for archived or created hours (or both).

Overall note: if we use "last viewed date" as a retention parameter, then videos can move _from our archived hours to our stored hours_. We also expect the "created" hours to occur over a year and they will flow into storage at an irregular pace. It's hard to account for these and some other factors (like, if we turn off automatic Zoom imports, how many hours will be created?) so we want breathing room under all the hours limits.

### Scenario 1: Save Money

We purchase no additional hours bundles. We have to turn off Zoom import, archive some (but not many) recently watched videos, and delete almost all of the videos that have not been watched.

In [194]:
total = data['hours'].sum()
zoom_import_percent = 0.2
created = ly_data[ly_data['Stream Source'] != 'Zoom']['hours'].sum() + zoom_import_percent * ly_data[ly_data['Stream Source'] == 'Zoom']['hours'].sum()
print('We would have {} created hours if we assume {}% of Zoom hours are manually imported.'.format(round(created, 2), zoom_import_percent * 100))
# we want incoming + most recently watched < 8,800
stored = created + watched_18_months['hours'].sum()
print('If we store any video watched in the last 18 months, combined with our created hours that is {} storage hours.'.format(round(stored, 2)))
watched_over_18_months = watched['hours'].sum() - watched_18_months['hours'].sum().round(2)
remainder =  watched_over_18_months + unwatched['hours'].sum()
non_user_non_zoom = unwatched[(unwatched['Stream Source'] != 'Zoom') & (unwatched['Root Folder'] != 'User')]['hours'].sum()
print('That leaves {} hours left to fill 4400 archived hours. We could prioritize the {} hours that are under non-User folders and from a non-Zoom source and the {} hours that had at least one view.'.format(round(remainder, 2), round(non_user_non_zoom, 2), round(watched_over_18_months, 2)))
archived = non_user_non_zoom + watched_over_18_months


We would have 2200.58 created hours if we assume 20.0% of Zoom hours are manually imported.
If store any video watched in the last 18 months, combined with our created hours that is 7035.73 storage hours.
That leaves 14791.98 hours left to fill 4400 archived hours. We could prioritize the 543.43 hours that are under non-User folders and from a non-Zoom source and the 935.24 hours that had at least one view.


In [195]:
pd.DataFrame({
    'Hours': ['Stored', 'Created', 'Archived', 'Deleted'],
    'Limit': [8800, 4400, 4400, 'n/a'],
    'CCA': [stored, created, archived, unwatched['hours'].sum() - 4400]
})

,Hours,Limit,CCA
0,Stored,8800,7035.733043
1,Created,4400,2200.581615
2,Archived,4400,1478.670788
3,Deleted,n/a,9456.741074


### Scenario 2: Save Content

We purchase one 50,000 hour bundle of archived hours. We have to turn off Zoom import, but we don't have to delete _anything_ (and probably won't for at least another year). The first half of the logic of the above scenario applies.

In [196]:
pd.DataFrame({
    'Hours': ['Stored', 'Created', 'Archived', 'Deleted'],
    'Limit': [8800, 4400, 54400, 'n/a'],
    'CCA': [stored, created, total - watched_18_months['hours'].sum(), 0]
})

,Hours,Limit,CCA
0,Stored,8800,7035.733043
1,Created,4400,2200.581615
2,Archived,54400,14791.980391
3,Deleted,n/a,0.000000


### Scenario 3: Save Zoom

We purchase one 4000 hour bundle of created hours. We created 6732 hours last year so we can retain the automatic Zoom import. We have _lots_ of content that either needs to be deleted or archived, though. This assumes the same amount of created content as last year.

Because our created hours _almost_ fill up the storage bucket on their own, we can only hold onto watched videos for a short amount of time. Videos watched in the last 6 months and incoming hours put us close to our stored hours bucket. We have to delete unwatched videos after an amount of time, but can store most watched videos from the past two years in the archive. Since the stored and archived hours run very close to their limits in this scenario, it may take continual monitoring and tweaking of retention policies.

In [198]:
created = ly_data['hours'].sum()
stored = created + watched_6_months['hours'].sum()
archived = watched['hours'].sum() - watched_6_months['hours'].sum()
pd.DataFrame({
    'Hours': ['Stored', 'Created', 'Archived', 'Deleted'],
    'Limit': [8800, 8400, 4400, 'n/a'],
    'CCA': [stored, created, archived, unwatched['hours'].sum()]
})

,Hours,Limit,CCA
0,Stored,8800,8494.737419
1,Created,8400,6732.910874
2,Archived,4400,4008.564200
3,Deleted,n/a,13856.741074


### Scenario 4: Save Both

We purchase one 50,000 hour bundle of archived hours _and_ one 4000 hour bundle of created hours. We still need to aggressively archive unwatched sessions because our created hours plus recently (<6 months) watched videos almost fill up the stored bucket, but we have space for _everything_ in the archive for a few years.

In [178]:
# stored & created are defined in the last scenario
archived = total - watched['hours'].sum()
pd.DataFrame({
    'Hours': ['Stored', 'Created', 'Archived', 'Deleted'],
    'Limit': [8800, 8400, 54400, 'n/a'],
    'CCA': [stored, created, archived, 0]
})

,Hours,Limit,CCA
0,Stored,8800,7035.733043
1,Created,8400,2200.581615
2,Archived,54400,13856.741074
3,Deleted,n/a,0.000000
